In [1]:
import pickle
import numpy as np
from keras import backend as K

participantCount = 32

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_peripheral_x():
    return data[:, :, 32:, :].reshape(-1, 8, 8064)

def get_eeg_x():
    return data[:, :, :32, :].reshape(-1, 32, 8064)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [2]:
from matplotlib import pyplot
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import Concatenate
from keras.layers import Layer
from keras.models import Model

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

class WeightedAverage(Layer):

    def __init__(self, n_output):
        super(WeightedAverage, self).__init__()
        self.W = tf.Variable(initial_value=tf.random.uniform(shape=[1,1,n_output], minval=0, maxval=1),
            trainable=True) # (1,1,n_inputs)

    def call(self, inputs):

        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim

        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)

def get_model():
    EEG_In = Input((8064,32))
    Per_In = Input((8064,8))
    EEG_1 = LSTM(20, input_dim=32)(EEG_In)
    Per_1 = LSTM(20, input_dim=8)(Per_In)
    EEG_2 = Dense(1, activation='relu')(EEG_1)
    Per_2 = Dense(1, activation='relu')(Per_1)

    x = [EEG_2, Per_2]
    W_Avg = WeightedAverage(n_output=len(x))(x)

    out = Dense(1, activation='sigmoid')(W_Avg)
    model = Model([EEG_In, Per_In], out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
    return model
    
X1 = np.moveaxis(get_eeg_x(), 1, 2)
X2 = np.moveaxis(get_peripheral_x(), 1, 2)

for i in [0, 1, 3]:
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    fscores = []
    for train, test in kfold.split(X1, Y):
        model = get_model()

        model.fit([X1[train], X2[train]], Y[train], validation_data=([X1[test], X2[test]], Y[test]), epochs=50, batch_size=128, verbose=2, shuffle=False)

        scores = model.evaluate([X1[test], X2[test]], Y[test], verbose=0)
        print("%s: %.2f%%, %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))
        cvscores.append(scores[1] * 100)
        fscores.append(scores[2] * 100)

    print("%.2f%% (+/- %.2f%%), %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores), np.mean(fscores), np.std(fscores)))    
    print("EMOTION", i)




EMOTION NUMBER 0
Epoch 1/50
8/8 - 9s - loss: 0.7685 - acc: 0.4795 - f1_m: 0.2438 - precision_m: 0.5208 - recall_m: 0.1751 - val_loss: 0.7742 - val_acc: 0.4492 - val_f1_m: 0.1543 - val_precision_m: 0.6786 - val_recall_m: 0.0891
Epoch 2/50
8/8 - 4s - loss: 0.7433 - acc: 0.4912 - f1_m: 0.2838 - precision_m: 0.6219 - recall_m: 0.2052 - val_loss: 0.7635 - val_acc: 0.4453 - val_f1_m: 0.1653 - val_precision_m: 0.6500 - val_recall_m: 0.0961
Epoch 3/50
8/8 - 4s - loss: 0.7283 - acc: 0.5068 - f1_m: 0.3218 - precision_m: 0.6468 - recall_m: 0.2368 - val_loss: 0.7516 - val_acc: 0.4414 - val_f1_m: 0.1470 - val_precision_m: 0.6846 - val_recall_m: 0.0836
Epoch 4/50
8/8 - 4s - loss: 0.7179 - acc: 0.5127 - f1_m: 0.3377 - precision_m: 0.6443 - recall_m: 0.2522 - val_loss: 0.7436 - val_acc: 0.4648 - val_f1_m: 0.2050 - val_precision_m: 0.7500 - val_recall_m: 0.1219
Epoch 5/50
8/8 - 4s - loss: 0.7091 - acc: 0.5273 - f1_m: 0.3615 - precision_m: 0.6705 - recall_m: 0.2741 - val_loss: 0.7387 - val_acc: 0.4766 -

Epoch 40/50
8/8 - 4s - loss: 0.5646 - acc: 0.7734 - f1_m: 0.8021 - precision_m: 0.7673 - recall_m: 0.8427 - val_loss: 0.7071 - val_acc: 0.5469 - val_f1_m: 0.5940 - val_precision_m: 0.6319 - val_recall_m: 0.5629
Epoch 41/50
8/8 - 4s - loss: 0.5605 - acc: 0.7676 - f1_m: 0.7971 - precision_m: 0.7615 - recall_m: 0.8386 - val_loss: 0.7085 - val_acc: 0.5508 - val_f1_m: 0.5930 - val_precision_m: 0.6372 - val_recall_m: 0.5559
Epoch 42/50
8/8 - 4s - loss: 0.5572 - acc: 0.7715 - f1_m: 0.8027 - precision_m: 0.7611 - recall_m: 0.8519 - val_loss: 0.7089 - val_acc: 0.5625 - val_f1_m: 0.6069 - val_precision_m: 0.6440 - val_recall_m: 0.5746
Epoch 43/50
8/8 - 4s - loss: 0.5530 - acc: 0.7705 - f1_m: 0.8016 - precision_m: 0.7624 - recall_m: 0.8486 - val_loss: 0.7093 - val_acc: 0.5703 - val_f1_m: 0.6159 - val_precision_m: 0.6482 - val_recall_m: 0.5871
Epoch 44/50
8/8 - 4s - loss: 0.5494 - acc: 0.7754 - f1_m: 0.8070 - precision_m: 0.7642 - recall_m: 0.8580 - val_loss: 0.7103 - val_acc: 0.5703 - val_f1_m: 0

Epoch 29/50
8/8 - 4s - loss: 0.6115 - acc: 0.7871 - f1_m: 0.8233 - precision_m: 0.7704 - recall_m: 0.8887 - val_loss: 0.6981 - val_acc: 0.5117 - val_f1_m: 0.5135 - val_precision_m: 0.5616 - val_recall_m: 0.4759
Epoch 30/50
8/8 - 4s - loss: 0.6063 - acc: 0.7881 - f1_m: 0.8239 - precision_m: 0.7726 - recall_m: 0.8875 - val_loss: 0.7001 - val_acc: 0.5117 - val_f1_m: 0.5132 - val_precision_m: 0.5611 - val_recall_m: 0.4749
Epoch 31/50
8/8 - 4s - loss: 0.6016 - acc: 0.7920 - f1_m: 0.8272 - precision_m: 0.7755 - recall_m: 0.8914 - val_loss: 0.7027 - val_acc: 0.5039 - val_f1_m: 0.5018 - val_precision_m: 0.5541 - val_recall_m: 0.4614
Epoch 32/50
8/8 - 4s - loss: 0.5959 - acc: 0.7949 - f1_m: 0.8290 - precision_m: 0.7788 - recall_m: 0.8912 - val_loss: 0.7051 - val_acc: 0.4922 - val_f1_m: 0.4800 - val_precision_m: 0.5441 - val_recall_m: 0.4335
Epoch 33/50
8/8 - 4s - loss: 0.5908 - acc: 0.7979 - f1_m: 0.8324 - precision_m: 0.7776 - recall_m: 0.8984 - val_loss: 0.7075 - val_acc: 0.4961 - val_f1_m: 0

Epoch 18/50
8/8 - 4s - loss: 0.6531 - acc: 0.6963 - f1_m: 0.7110 - precision_m: 0.7793 - recall_m: 0.6591 - val_loss: 0.7081 - val_acc: 0.4805 - val_f1_m: 0.5162 - val_precision_m: 0.5116 - val_recall_m: 0.5433
Epoch 19/50
8/8 - 4s - loss: 0.6499 - acc: 0.6973 - f1_m: 0.7144 - precision_m: 0.7757 - recall_m: 0.6675 - val_loss: 0.7080 - val_acc: 0.4648 - val_f1_m: 0.5087 - val_precision_m: 0.4989 - val_recall_m: 0.5433
Epoch 20/50
8/8 - 4s - loss: 0.6467 - acc: 0.7012 - f1_m: 0.7235 - precision_m: 0.7695 - recall_m: 0.6881 - val_loss: 0.7079 - val_acc: 0.4453 - val_f1_m: 0.4929 - val_precision_m: 0.4826 - val_recall_m: 0.5306
Epoch 21/50
8/8 - 4s - loss: 0.6435 - acc: 0.7070 - f1_m: 0.7298 - precision_m: 0.7763 - recall_m: 0.6937 - val_loss: 0.7076 - val_acc: 0.4570 - val_f1_m: 0.5088 - val_precision_m: 0.4971 - val_recall_m: 0.5524
Epoch 22/50
8/8 - 4s - loss: 0.6397 - acc: 0.7207 - f1_m: 0.7441 - precision_m: 0.7855 - recall_m: 0.7140 - val_loss: 0.7081 - val_acc: 0.4531 - val_f1_m: 0

Epoch 7/50
8/8 - 4s - loss: 0.6425 - acc: 0.5811 - f1_m: 0.6718 - precision_m: 0.6009 - recall_m: 0.7690 - val_loss: 0.7002 - val_acc: 0.5156 - val_f1_m: 0.6032 - val_precision_m: 0.5624 - val_recall_m: 0.6528
Epoch 8/50
8/8 - 4s - loss: 0.6365 - acc: 0.5918 - f1_m: 0.6790 - precision_m: 0.6103 - recall_m: 0.7745 - val_loss: 0.6994 - val_acc: 0.5273 - val_f1_m: 0.6139 - val_precision_m: 0.5705 - val_recall_m: 0.6667
Epoch 9/50
8/8 - 4s - loss: 0.6316 - acc: 0.6006 - f1_m: 0.6848 - precision_m: 0.6172 - recall_m: 0.7779 - val_loss: 0.7001 - val_acc: 0.5273 - val_f1_m: 0.6161 - val_precision_m: 0.5692 - val_recall_m: 0.6736
Epoch 10/50
8/8 - 4s - loss: 0.6257 - acc: 0.6104 - f1_m: 0.6917 - precision_m: 0.6258 - recall_m: 0.7830 - val_loss: 0.7004 - val_acc: 0.5273 - val_f1_m: 0.6131 - val_precision_m: 0.5702 - val_recall_m: 0.6667
Epoch 11/50
8/8 - 4s - loss: 0.6206 - acc: 0.6250 - f1_m: 0.7018 - precision_m: 0.6355 - recall_m: 0.7916 - val_loss: 0.7009 - val_acc: 0.5234 - val_f1_m: 0.60

...

Epoch 36/50
8/8 - 4s - loss: 0.5906 - acc: 0.7969 - f1_m: 0.8607 - precision_m: 0.7732 - recall_m: 0.9731 - val_loss: 0.6991 - val_acc: 0.6367 - val_f1_m: 0.7184 - val_precision_m: 0.7382 - val_recall_m: 0.7021
Epoch 37/50
8/8 - 4s - loss: 0.5875 - acc: 0.7969 - f1_m: 0.8610 - precision_m: 0.7720 - recall_m: 0.9760 - val_loss: 0.7006 - val_acc: 0.6406 - val_f1_m: 0.7211 - val_precision_m: 0.7389 - val_recall_m: 0.7067
Epoch 38/50
8/8 - 4s - loss: 0.5841 - acc: 0.7979 - f1_m: 0.8621 - precision_m: 0.7718 - recall_m: 0.9793 - val_loss: 0.7028 - val_acc: 0.6328 - val_f1_m: 0.7158 - val_precision_m: 0.7375 - val_recall_m: 0.6974
Epoch 39/50
8/8 - 4s - loss: 0.5808 - acc: 0.8037 - f1_m: 0.8659 - precision_m: 0.7759 - recall_m: 0.9821 - val_loss: 0.7051 - val_acc: 0.6367 - val_f1_m: 0.7184 - val_precision_m: 0.7382 - val_recall_m: 0.7021
Epoch 40/50
8/8 - 4s - loss: 0.5774 - acc: 0.8037 - f1_m: 0.8657 - precision_m: 0.7773 - recall_m: 0.9797 - val_loss: 0.7082 - val_acc: 0.6289 - val_f1_m: 0

Epoch 25/50
8/8 - 4s - loss: 0.4745 - acc: 0.6807 - f1_m: 0.8065 - precision_m: 0.6807 - recall_m: 1.0000 - val_loss: 0.6785 - val_acc: 0.6250 - val_f1_m: 0.7690 - val_precision_m: 0.6250 - val_recall_m: 1.0000
Epoch 26/50
8/8 - 4s - loss: 0.4676 - acc: 0.6807 - f1_m: 0.8065 - precision_m: 0.6807 - recall_m: 1.0000 - val_loss: 0.6785 - val_acc: 0.6250 - val_f1_m: 0.7690 - val_precision_m: 0.6250 - val_recall_m: 1.0000
Epoch 27/50
8/8 - 4s - loss: 0.4605 - acc: 0.6807 - f1_m: 0.8065 - precision_m: 0.6807 - recall_m: 1.0000 - val_loss: 0.6793 - val_acc: 0.6250 - val_f1_m: 0.7690 - val_precision_m: 0.6250 - val_recall_m: 1.0000
Epoch 28/50
8/8 - 4s - loss: 0.4539 - acc: 0.6807 - f1_m: 0.8065 - precision_m: 0.6807 - recall_m: 1.0000 - val_loss: 0.6828 - val_acc: 0.6250 - val_f1_m: 0.7690 - val_precision_m: 0.6250 - val_recall_m: 1.0000
Epoch 29/50
8/8 - 4s - loss: 0.4479 - acc: 0.6807 - f1_m: 0.8065 - precision_m: 0.6807 - recall_m: 1.0000 - val_loss: 0.6850 - val_acc: 0.6250 - val_f1_m: 0

Epoch 14/50
8/8 - 4s - loss: 0.6007 - acc: 0.6787 - f1_m: 0.8051 - precision_m: 0.6787 - recall_m: 1.0000 - val_loss: 0.6548 - val_acc: 0.6328 - val_f1_m: 0.7744 - val_precision_m: 0.6328 - val_recall_m: 1.0000
Epoch 15/50
8/8 - 4s - loss: 0.5934 - acc: 0.6787 - f1_m: 0.8051 - precision_m: 0.6787 - recall_m: 1.0000 - val_loss: 0.6556 - val_acc: 0.6328 - val_f1_m: 0.7744 - val_precision_m: 0.6328 - val_recall_m: 1.0000
Epoch 16/50
8/8 - 4s - loss: 0.5866 - acc: 0.6787 - f1_m: 0.8051 - precision_m: 0.6787 - recall_m: 1.0000 - val_loss: 0.6562 - val_acc: 0.6328 - val_f1_m: 0.7744 - val_precision_m: 0.6328 - val_recall_m: 1.0000
Epoch 17/50
8/8 - 4s - loss: 0.5789 - acc: 0.6787 - f1_m: 0.8051 - precision_m: 0.6787 - recall_m: 1.0000 - val_loss: 0.6562 - val_acc: 0.6328 - val_f1_m: 0.7744 - val_precision_m: 0.6328 - val_recall_m: 1.0000
Epoch 18/50
8/8 - 4s - loss: 0.5719 - acc: 0.6787 - f1_m: 0.8051 - precision_m: 0.6787 - recall_m: 1.0000 - val_loss: 0.6562 - val_acc: 0.6328 - val_f1_m: 0

Epoch 3/50
8/8 - 4s - loss: 0.7154 - acc: 0.5254 - f1_m: 0.5678 - precision_m: 0.7463 - recall_m: 0.4674 - val_loss: 0.7263 - val_acc: 0.4492 - val_f1_m: 0.4561 - val_precision_m: 0.6073 - val_recall_m: 0.3713
Epoch 4/50
8/8 - 4s - loss: 0.7081 - acc: 0.5400 - f1_m: 0.5997 - precision_m: 0.7314 - recall_m: 0.5173 - val_loss: 0.7232 - val_acc: 0.4414 - val_f1_m: 0.4644 - val_precision_m: 0.5897 - val_recall_m: 0.3881
Epoch 5/50
8/8 - 4s - loss: 0.7036 - acc: 0.5674 - f1_m: 0.6281 - precision_m: 0.7461 - recall_m: 0.5505 - val_loss: 0.7205 - val_acc: 0.4570 - val_f1_m: 0.4914 - val_precision_m: 0.6022 - val_recall_m: 0.4215
Epoch 6/50
8/8 - 4s - loss: 0.6984 - acc: 0.5986 - f1_m: 0.6657 - precision_m: 0.7569 - recall_m: 0.6011 - val_loss: 0.7181 - val_acc: 0.4766 - val_f1_m: 0.5146 - val_precision_m: 0.6239 - val_recall_m: 0.4453
Epoch 7/50
8/8 - 4s - loss: 0.6943 - acc: 0.6094 - f1_m: 0.6816 - precision_m: 0.7503 - recall_m: 0.6293 - val_loss: 0.7166 - val_acc: 0.4844 - val_f1_m: 0.5379

Epoch 42/50
8/8 - 4s - loss: 0.5745 - acc: 0.8584 - f1_m: 0.9014 - precision_m: 0.8379 - recall_m: 0.9770 - val_loss: 0.7117 - val_acc: 0.4961 - val_f1_m: 0.5954 - val_precision_m: 0.6070 - val_recall_m: 0.6000
Epoch 43/50
8/8 - 4s - loss: 0.5701 - acc: 0.8613 - f1_m: 0.9030 - precision_m: 0.8408 - recall_m: 0.9770 - val_loss: 0.7127 - val_acc: 0.4922 - val_f1_m: 0.5908 - val_precision_m: 0.6044 - val_recall_m: 0.5944
Epoch 44/50
8/8 - 4s - loss: 0.5664 - acc: 0.8594 - f1_m: 0.9007 - precision_m: 0.8444 - recall_m: 0.9669 - val_loss: 0.7148 - val_acc: 0.4883 - val_f1_m: 0.5807 - val_precision_m: 0.6027 - val_recall_m: 0.5776
Epoch 45/50
8/8 - 4s - loss: 0.5629 - acc: 0.8604 - f1_m: 0.9015 - precision_m: 0.8450 - recall_m: 0.9681 - val_loss: 0.7167 - val_acc: 0.4883 - val_f1_m: 0.5779 - val_precision_m: 0.6032 - val_recall_m: 0.5719
Epoch 46/50
8/8 - 4s - loss: 0.5587 - acc: 0.8643 - f1_m: 0.9043 - precision_m: 0.8455 - recall_m: 0.9736 - val_loss: 0.7205 - val_acc: 0.4844 - val_f1_m: 0